In [1]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader 
from torch.utils.tensorboard import SummaryWriter

In [2]:
class CNN(nn.Module):
    def __init__(self, in_channels, num_classes):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=in_channels, out_channels=8, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(16 * 7 * 7, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc1(x)
        return x

In [3]:
device = "cuda" if torch.cuda.is_available else "cpu"

In [4]:
lr = 0.001
in_channel = 1
num_classes = 10
batch_size = 10
num_epochs = 5

In [5]:
train_dataset = datasets.MNIST(root = "data/mnist", train = True, transform=transforms.ToTensor(), download = True)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

model = CNN(in_channels=in_channel, num_classes=num_classes)
model.to(device)

CNN(
  (conv1): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=784, out_features=10, bias=True)
)

In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = lr, weight_decay=0.0)

In [7]:
write = SummaryWriter()
step = 0

In [8]:
for epoch in range(num_epochs):
    losses = []
    accuracies = []
    for batch, (data, label) in enumerate(train_loader):
        data = data.to(device)
        label = label.to(device)

        y_pred = model(data)
        loss = loss_fn(y_pred, label)
        losses.append(loss.item())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        _, prediction = y_pred.max(1)
        num_correct = (prediction == label).sum()
        running_acc = float(num_correct)/float(data.shape[0])

        write.add_scalar("Training loss", loss, global_step=step)
        step += 1

        write.add_scalar("Training accuracy", running_acc, global_step=step)
        